In [1]:
!pip -q install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 46.2 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk, string
from google.colab import drive
import torch
import tqdm
from keras_preprocessing.sequence import pad_sequences
from transformers import BertTokenizer,  AutoModelForSequenceClassification
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize


nltk.download('punkt') # if necessary...


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
pd.set_option('display.max_colwidth', None)
data_path = "/content/drive/MyDrive/Data/combined.csv"
df = pd.read_csv(data_path, index_col = 0)
print(df.shape)

(49801, 1)


In [5]:
def preprocess_data(data_path, sample_size):

  # Read the data from specific path
  data = pd.read_csv(data_path, index_col=0)
  # data = pd.read_csv(data_path, low_memory=False, quoting=3, error_bad_lines=False)

  # Drop articles without Abstract
  data = data.dropna(subset = ['abstract']).reset_index(drop = True)

  # Get "sample_size" random articles
  # data = data.sample(sample_size)[['abstract']]

  return data
  
source_data = preprocess_data(data_path, 44000)
print(source_data.shape)
display(source_data.head())
#source_data = source_data['abstract'].tolist()

(49801, 1)


,abstract
0,"Transformers have shown great potential in computer vision tasks. A common\nbelief is their attention-based token mixer module contributes most to their\ncompetence. However, recent works show the attention-based module in\ntransformers can be replaced by spatial MLPs and the resulted models still\nperform quite well. Based on this observation, we hypothesize that the general\narchitecture of the transformers, instead of the specific token mixer module,\nis more essential to the model's performance. To verify this, we deliberately\nreplace the attention module in transformers with an embarrassingly simple\nspatial pooling operator to conduct only the most basic token mixing.\nSurprisingly, we observe that the derived model, termed as PoolFormer, achieves\ncompetitive performance on multiple computer vision tasks. For example, on\nImageNet-1K, PoolFormer achieves 82.1% top-1 accuracy, surpassing well-tuned\nvision transformer/MLP-like baselines DeiT-B/ResMLP-B24 by 0.3%/1.1% accuracy\nwith 35%/52% fewer parameters and 48%/60% fewer MACs. The effectiveness of\nPoolFormer verifies our hypothesis and urges us to initiate the concept of\n""MetaFormer"", a general architecture abstracted from transformers without\nspecifying the token mixer. Based on the extensive experiments, we argue that\nMetaFormer is the key player in achieving superior results for recent\ntransformer and MLP-like models on vision tasks. This work calls for more\nfuture research dedicated to improving MetaFormer instead of focusing on the\ntoken mixer modules. Additionally, our proposed PoolFormer could serve as a\nstarting baseline for future MetaFormer architecture design. Code is available\nat https://github.com/sail-sg/poolformer"
1,"A critical aspect of reliable communication involves the design of codes that\nallow transmissions to be robustly and computationally efficiently decoded\nunder noisy conditions. Advances in the design of reliable codes have been\ndriven by coding theory and have been sporadic. Recently, it is shown that\nchannel codes that are comparable to modern codes can be learned solely via\ndeep learning. In particular, Turbo Autoencoder (TURBOAE), introduced by Jiang\net al., is shown to achieve the reliability of Turbo codes for Additive White\nGaussian Noise channels. In this paper, we focus on applying the idea of\nTURBOAE to various practical channels, such as fading channels and chirp noise\nchannels. We introduce TURBOAE-TI, a novel neural architecture that combines\nTURBOAE with a trainable interleaver design. We develop a carefully-designed\ntraining procedure and a novel interleaver penalty function that are crucial in\nlearning the interleaver and TURBOAE jointly. We demonstrate that TURBOAE-TI\noutperforms TURBOAE and LTE Turbo codes for several channels of interest. We\nalso provide interpretation analysis to better understand TURBOAE-TI."
2,"Point defects are responsible for a wide range of optoelectronic properties\nin materials, making it crucial to engineer their concentrations for novel\nmaterials design. However, considering the plethora of defects in co-doped\nsemiconducting and dielectric materials and the dependence of defect formation\nenergies on heat treatment parameters, process design based on an experimental\ntrial and error approach is not an efficient strategy. This makes it necessary\nto explore computational pathways for predicting defect equilibria during heat\ntreatments. The accumulated experimental knowledge on defect transformations in\ndiamond is unparalleled. Therefore, diamond is an excellent material for\nbenchmarking computational approaches. By considering nitrogen, hydrogen, and\nsilicon doped diamond as a model system, we have investigated the pressure\ndependence of defect formation energies and calculated the defect equilibria\nduring heat treatment of diamond through ab-initio calculations. We have\nplotted monolithic-Kr\""oger-Vink diagrams for various defects, representing\nd

In [6]:
# Initialize the stemmer and tokenizer
stemmer = PorterStemmer()
tokenizer = word_tokenize
# Create a TfidfVectorizer object to convert the text data into numerical features
tfidf = TfidfVectorizer()

# Preprocess the text data
def preprocess_text(text):
    # Tokenize the text
    tokens = tokenizer(text)
    
    # Stem each token
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    
    # Join the stemmed tokens back into a single string
    stemmed_text = ' '.join(stemmed_tokens)
    
    return stemmed_text

In [7]:
# Preprocess the text data of each research paper
preprocessed_data = [preprocess_text(text) for text in source_data['abstract'].tolist()]
training_data = tfidf.fit_transform(preprocessed_data)


In [8]:
# Checking preprocessed data
display(preprocessed_data[:5])

["transform have shown great potenti in comput vision task . a common belief is their attention-bas token mixer modul contribut most to their compet . howev , recent work show the attention-bas modul in transform can be replac by spatial mlp and the result model still perform quit well . base on thi observ , we hypothes that the gener architectur of the transform , instead of the specif token mixer modul , is more essenti to the model 's perform . to verifi thi , we deliber replac the attent modul in transform with an embarrassingli simpl spatial pool oper to conduct onli the most basic token mix . surprisingli , we observ that the deriv model , term as poolform , achiev competit perform on multipl comput vision task . for exampl , on imagenet-1k , poolform achiev 82.1 % top-1 accuraci , surpass well-tun vision transformer/mlp-lik baselin deit-b/resmlp-b24 by 0.3 % /1.1 % accuraci with 35 % /52 % fewer paramet and 48 % /60 % fewer mac . the effect of poolform verifi our hypothesi and u

In [9]:
def process_document(text):
    """
    Create a vector for given text and adjust it for cosine similarity search
    """
    text_vect = create_vector_from_text(tokenizer, model, text)
    text_vect = np.array(text_vect)
    text_vect = text_vect.reshape(1, -1)

    return text_vect

def create_vector_from_text(tokenizer, model, text, MAX_LEN = 510):
    
    input_ids = tokenizer.encode(
                        text, 
                        add_special_tokens = True, 
                        max_length = MAX_LEN,                           
                   )    

    results = pad_sequences([input_ids], maxlen=MAX_LEN, dtype="long", 
                              truncating="post", padding="post")
    
    # Remove the outer list.
    input_ids = results[0]

    # Create attention masks    
    attention_mask = [int(i>0) for i in input_ids]
    
    # Convert to tensors.
    input_ids = torch.tensor(input_ids)
    attention_mask = torch.tensor(attention_mask)

    # Add an extra dimension for the "batch" (even though there is only one 
    # input in this batch.)
    input_ids = input_ids.unsqueeze(0)
    attention_mask = attention_mask.unsqueeze(0)
    
    # Put the model in "evaluation" mode, meaning feed-forward operation.
    model.eval()
    
    # Run the text through BERT, and collect all of the hidden states produced
    # from all 12 layers. 
    with torch.no_grad():        
        logits, encoded_layers = model(
                                    input_ids = input_ids, 
                                    token_type_ids = None, 
                                    attention_mask = attention_mask,
                                    return_dict=False)

    layer_i = 12 # The last BERT layer before the classifier.
    batch_i = 0 # Only one input in the batch.
    token_i = 0 # The first token, corresponding to [CLS]
        
    # Extract the embedding.
    vector = encoded_layers[layer_i][batch_i][token_i]

    # Move to the CPU and convert to numpy ndarray.
    vector = vector.detach().cpu().numpy()

    return(vector)

def create_vector_index(data):
    
    # The list of all the vectors
    vectors = []
    
    # Get overall text data
    source_data = data.abstract.values
    
    # Loop over all the comment and get the embeddings
    for text in tqdm(source_data):
        
        # Get the embedding 
        vector = create_vector_from_text(tokenizer, model, text)
        
        #add it to the list
        vectors.append(vector)
    
    data["vectors"] = vectors
    data["vectors"] = data["vectors"].apply(lambda emb: np.array(emb))
    data["vectors"] = data["vectors"].apply(lambda emb: emb.reshape(1, -1))
    
    return data

In [ ]:
vector_index = create_vector_index(source_data)